### Producing intensity variations using LLMs

Input: a baseline sentence; Output: variations of this sentence where a target word is more or less intense


In [20]:
from dotenv import load_dotenv
load_dotenv()

from dataclasses import dataclass
import simplemind as sm

TARGET_WORD = 'anxiety'

# can be changed to gemini, see https://pypi.org/project/simplemind/
PROVIDER = "openai"
MODEL = "gpt-4o"

Setup examples to inject in the prompt

In [21]:
@dataclass
class Example:
    baseline: str
    more_intense: str
    less_intense: str

    def format_for_prompt(self):
        return f"""<baseline>
{self.baseline}
</baseline>
<increased {TARGET_WORD}>
{self.more_intense}
</increased {TARGET_WORD}>
<decreased {TARGET_WORD}>
{self.less_intense}
</decreased {TARGET_WORD}>
"""

EXAMPLES = [
    Example(
        baseline="In a 24-yr-old female patient with a 12-yr history of kleptomania, it appeared that the behavior was maintained because it reduced anxiety in relevant situations.",
        more_intense="In a 24-yr-old female patient with a 12-yr history of kleptomania, it appeared that the behavior was maintained because it alleviated intense and overwhelming anxiety that pervaded her daily life.",
        less_intense="In a 24-yr-old female patient with a 12-yr history of kleptomania, it appeared that the behavior was maintained because it relieved mild, occasional anxiety in certain situations.",
    ),
    Example(
        baseline="Anxiety, insecurity, and lack of skill in establishing appropriate relationships prevented workers from obtaining adequate bases for evaluation and for effectuation of treatment.",
        more_intense="Intense anxiety, profound insecurity, and a lack of skill in establishing appropriate relationships prevented workers from obtaining adequate bases for evaluation and for effectuation of treatment.",
        less_intense="Mild anxiety, occasional insecurity, and a lack of skill in establishing appropriate relationships prevented workers from obtaining adequate bases for evaluation and for effectuation of treatment.",
    ),
    Example(
        baseline="The relationship between self-reported fear and anxiety was examined in a large sample of normal Australian children and adolescents.",
        more_intense="The relationship between deeply rooted fear and severe anxiety was examined in a large sample of Australian children and adolescents, revealing strong emotional undercurrents.",
        less_intense="The relationship between mild fear and occasional anxiety was examined in a large sample of Australian children and adolescents.",
    )
]

PROMPT_INTRO = f"""In psychology lexicographic research, we define "intensity" as the "extent to which a word refers to more emotionally or referentially intense phenomena". Here we study the intensity of the word "{TARGET_WORD}"
You will be given a sentence with the word "{TARGET_WORD}" in it. You will then be asked to write two new sentences: one where the word "{TARGET_WORD}" is more intense, and one where it is less intense.

"""


In [22]:
@dataclass
class SentenceToModify:
    text: str

    def get_prompt(self):
        prompt = PROMPT_INTRO
        for example in EXAMPLES:
            prompt += example.format_for_prompt()
            prompt += "\n\n"
        
        prompt += f"""<baseline>
{self.text}
</baseline>
"""
        return prompt
    
    def parse_response(self, response: str):
        # get the sentences inside <more {TARGET_WORD}> and <less {TARGET_WORD}>
        increased_variation = response.split(f"<increased {TARGET_WORD}>")[1].split(f"</increased {TARGET_WORD}>")[0].strip()
        decreased_variation = response.split(f"<decreased {TARGET_WORD}>")[1].split(f"</decreased {TARGET_WORD}>")[0].strip()
        return increased_variation, decreased_variation

    def get_variations(self) -> list[str]:
        """ Returns a list of two strings: one where the TARGET_WORD is more intense, and one where it is less intense """
        assert TARGET_WORD in self.text, f"TARGET_WORD {TARGET_WORD} not found in text"
        prompt = self.get_prompt()
        res = sm.generate_text(prompt=prompt, llm_provider=PROVIDER, llm_model=MODEL)
        return self.parse_response(res)


In [23]:
sentence = SentenceToModify(text="The anxiety caused by impending surgery left her feeling overwhelmed and vulnerable, affecting her daily life.")
sentence.get_variations()


('The profound and crippling anxiety caused by impending surgery consumed her, leaving her feeling completely overwhelmed and vulnerable, and severely impacting her daily life.',
 'The mild anxiety caused by impending surgery left her feeling somewhat concerned and vulnerable, but only slightly affected her daily life.')